# Works (except for delete synchronization)

In [1]:
from scipy.io.wavfile import read 
from os import  listdir,rename,remove,makedirs,path
from os.path import exists
from scipy.signal import spectrogram
from pydub import AudioSegment
from time import time, sleep
from math import sqrt, log
from numpy import int16, average, save, load
from _thread import start_new_thread
from tkinter.filedialog import askdirectory
from sys import exc_info
from subprocess import check_output,call
from mutagen.id3 import ID3,TPE1,TIT2,TCON,ID3NoHeaderError
import tkinter.ttk

# סינכרון עם הטלפון

In [2]:
def trans(directory):
    phones=[]                                                                                           #רשימת השירים על הטלפון
                       
       
              #רשימת השירים שיש בטלפון
    def phone_songs(phones):        
        try:
            garbage_phone = check_output("adb shell ls -n '/sdcard/Music'",encoding="850")
            for file_name in garbage_phone.splitlines():
                if (file_name[-4:].lower()=='.mp3'):                                                      #אם הקובץ הוא שיר
                    try:
                        file_name.index('  ')
                        phones.append([file_name.split(' ')[5],' '.join(file_name.split(' ')[8:])])
                    except:
                        phones.append([file_name.split(' ')[4],' '.join(file_name.split(' ')[7:])])                    
        except:
            print(exc_info()[1])
            pass
        save('old_songs', phones)
        return(phones)
    
                
            #מוחק שירים שנמצאים שלא נמצאים בטלפון ונמצאים במחשב מהמחשב (שנמחקו ממנו
    try:                                #יוזר ישן
        1/0
    except:                             #יוזר חדש
        phone_songs(phones)
        pass

    
            #מעביר שירים חדשים לטלפון
    for file_name in listdir(directory):
        if (file_name[-4:].lower()=='.mp3'):                                                     #אם הקובץ הוא שיר
            k=1                                                                                  #לא נמצא קובץ באותו המשקל
            p=0                                                                                  #מספר שורה
            while(k==1 and p<len(phones)):
                if (phones[p][1].lower()==file_name.lower()):
                    k=0                                                                          #נמצא קובץ באותו המשקל
                p+=1
            if (k==1):
                p=0                                                                              #מספר שורה
                while(k==1 and p<len(phones)):
                    if (int(phones[p][0])==path.getsize(directory+'/'+file_name)):
                        k=0                                                                      #נמצא קובץ באותו המשקל
                    p+=1
                if (k==1):
                    a=check_output('adb push "'+directory+'/'+file_name+'" /sdcard/Music')
                    print('pushing:',file_name,' ', path.getsize(directory+'/'+file_name))
                            
                    
                    
            #refresh the connection
    phone_name=check_output('adb devices').decode('utf-8').splitlines()[1]
    phone_name=phone_name[:phone_name.index('\t')]
    check_output('adb -s '+phone_name+' reconnect')

# עיבוד השמע

In [3]:
def processing(file_name,group,group_name,weights,directory):
    AudioSegment.from_mp3(directory+'/'+file_name).export(directory+'/'+file_name[:-4]+'.wav',format="wav")
    sample_rate, samples = read(directory+'/'+file_name[:-4]+'.wav')
    remove(directory+'/'+file_name[:-4]+'.wav')
    try:
        samples=int16(samples[:,0]/2+samples[:,1]/2)     #אם השיר הוא סטריאו תהפוך אותו למונו
    except:
        pass                                                   #אם השיר הוא מונו, תמשיך
    timegrouping=2048                                          #מספר המדידות ביחידת זמן של הספקטורגרם
    freqgroups=(92681.90003/sqrt(timegrouping))                #מספר יחידות התדירות בספקטורגרם
    f, t, Sxx = spectrogram(samples,sample_rate,nperseg=timegrouping,nfft=freqgroups)
    BPMin=70.5
    BPMax=2*BPMin
    Frange=[0,80,250,500,2000,4000,6000,max(f)]
    BPMs=[]
    Energies=[]
    for delay in range(int(round(60/(BPMax*(t[1]-t[0]))-0.5)),int(round(60/(BPMin*(t[1]-t[0]))-0.5))):#בעבור מספר קצבים שונים
        Energies.append((Sxx[int(round(Frange[-2]/(f[1]-f[0]))):int(round(Frange[-1]/(f[1]-f[0]))),:len(t)-delay]*
            Sxx[int(round(Frange[-2]/(f[1]-f[0]))):int(round(Frange[-1]/(f[1]-f[0]))),delay:len(t)]).sum())
                                         #תוסיף לרשימת האנרגיות את מכפלת האמפליטודה, בעבור התדרים הגבוהים, באמפליטודה בדיליי
        BPMs.append(60/t[delay])
    initial_BPM=BPMs[Energies.index(max(Energies))]            #התדירות בה המכפלה יצאה הגדולה ביותר בתחום התדר הגבוה
    timegrouping=3276800/initial_BPM**2
    freqgroups=(92681.90003/sqrt(timegrouping))
    HPF=initial_BPM-initial_BPM/8
    LPF=initial_BPM+initial_BPM/8
    Energies=[]
    BPM=[]
    for i in range(len(Frange)-1):                             #בעבור כל תחומי התדר השונים
        BPMs=[]
        Energies=[]
        for delay in range(int(round(60/(LPF*(t[1]-t[0]))-0.5)),int(round(60/(HPF*(t[1]-t[0]))-0.5))):#בעבור מספר קצבים שונים
            Energies.append((Sxx[int(round(Frange[i]/(f[1]-f[0]))):int(round(Frange[i+1]/(f[1]-f[0]))),:len(t)-delay]*
                Sxx[int(round(Frange[i]/(f[1]-f[0]))):int(round(Frange[i+1]/(f[1]-f[0]))),delay:len(t)]).sum())
                                          #תוסיף לרשימת האנרגיות את מכפלת האמפליטודה, בעבור תחום התדר הרלוונטי, באמפליטודה בדיליי
            BPMs.append(60/t[delay])
        BPM.append(BPMs[Energies.index(max(Energies))])
    result=average(BPM,weights=weights)                  #התדירות המשוערת
    if (BPMin>result):                                         #אם התדירות קטנה מדי
        result=result*2
    elif (result>BPMax):                                       #אם התדירות גדולה מדי
        result=result/2
    for i in range(len(group)-1):                              #הלולאה מה משנים את הז'אנר
        if (group[i]<=result<group[i+1]):
            audio=ID3(str(directory+"/"+file_name))
            audio["TCON"]=TCON(text=group_name[i])
            audio.save()             
            break
    if (result==group[-1]):
        audio=ID3(str(directory+"/"+file_name))
        audio["TCON"]=TCON(text=group_name[-1])
        audio.save()             

# הרצת עיבוד השמע עבור כל השירים

In [4]:
root=tkinter.Tk()
root.title("Songs BPM Classifier")
root.geometry('400x200')
root.progress=tkinter.ttk.Progressbar(root,orient="horizontal",length=200,mode="determinate")
label1=tkinter.Label(root,text="Please choose unclassified directory")
def behind():
    sleep(0.001)
    root.update()
def update(directory,groups):
    try:
        group=[70.5]
        for i in range(1,groups):
            group.append(int(70.5**(log(141,70.5)**(i/groups))))                 #ערכי תחומי התיקיות
        group.append(141)
        if (groups==3):
            group_name=['Chillz','Casual','Boost']
        elif (groups==4):
            group_name=['Chillz','Casual','Warm Up','Boost']
        else:
            group_name=['Sleep','Chillz','Casual','Warm Up','Boost']
        done=0
        total=0
        for file_name in listdir(directory):                #בודק כמה תמונות יש בתקייה
            if (file_name[-4:].lower()=='.mp3'):
                try:
                    audio=ID3(str(directory+"/"+file_name))
                    audio["TCON"]
                except:
                    total+=1     
        if (total==0):                                          #אם התוכנה כבר קיטלגה את כל השירים
            label1['text']="Syncing songs"
            trans(directory)
            label1['text']="Done"
        else:                                                   #אם ישנם שירים חסרי ז'אנר שצריך לקטלג
            start=time()
            weights=[6.80334738e-24,1.84428939e-23,3.52887836e-23,2.11613410e-23,2.59560180e-22,1.48475353e-22,2.51389276e-23]
            for file_name in listdir(directory):                #בעבור כל קובץ בתקייה
                if (file_name[-4:].lower()=='.mp3'):            #אם הקובץ הוא שיר
                    try:                                        #אם לשיר אין ז'אנר 
                        audio=ID3(str(directory+"/"+file_name))
                        audio["TCON"]                           
                    except:
                        processing(file_name,group,group_name,weights,directory)
                        done+=1
                        root.progress['value']=done/total*100
                        if (60<int((time()-start)/(done/total)-(time()-start))):
                            label1['text']="Estimated time: "+str(int(((time()-start)/(done/total)-(time()-start))/60))+" minutes & "+str(int(((time()-start)/(done/total)-(time()-start))%60))+" seconds   "
                        else:
                            label1['text']="Estimated time: "+str(int((time()-start)/(done/total)-(time()-start)))+" seconds   "
                    root.update_idletasks()                     #מעדכן את פלט התוכנה
            label1['text']="Syncing songs"
            trans(directory)
            label1['text']="Done"
    except:
        label1['text']=exc_info()[1]
def helloCallBack():
    directory=askdirectory()
    groups=scale1.get()
    start_new_thread(update,(directory,groups))
    start_new_thread(behind,())
    label1['text']="runing..."
tkinter.Button(text ="Please choose input directory", command = helloCallBack).pack()
tkinter.Label(root, text="Number of songs groups:").pack()
scale1=tkinter.Scale(root,from_=2.0, to_=6.0, orient="horizontal", length=200)
scale1.set(4)
scale1.pack()
root.progress.pack()
label1.pack()
tkinter.Label(root, text="\n\nAll rights reserved to Gon Eyal©").pack()
root.mainloop()

Command 'adb shell ls -n '/sdcard/Music'' returned non-zero exit status 1.


In [5]:
try:
    d="C:/Users/Gon/Downloads"
    a=0
    for file_name in listdir(d):                
        if (file_name[-4:].lower()=='.mp3'):            
            try:                                      
                audio=ID3(str(d+"/"+file_name))
                audio["TCON"]                           
            except:
                a+=1
except:print('1')
print(a)

0
